<a href="https://colab.research.google.com/github/SnyYmc2023/Semboller/blob/main/YazalimBiraz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Kodların tamamını çalıştırmak için ctrl+F9 veya Çalışma zamanı -> Tümünü çalıştır.**

In [ ]:
# Gerekli kütüphanelerin kurulumu. Mutlaka ilk bunu çalıştırın!
!pip install git+https://github.com/rongardF/tvdatafeed tradingview-screener ta




---

**5-13-21 üstünde koşulunu sağlayan yani potansiyel bir trend mi :) çok fazla sonuç gelecektir.... inandırıcı mı ilerleyen günlerde izleyelim. Son fiyat ve tarih bilgisi var sonuçlarda.**

In [8]:
import numpy as np
import pandas as pd
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
import warnings
import logging
import time

# Uyarıları ve log ayarlarını yapılandırma
warnings.simplefilter(action='ignore')
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

# TradingView verilerini indirmek için kimlik bilgileriyle oturum açma
tv = TvDatafeed(username='your_username', password='your_password')

def calculate_ema(data):
    """
    Verilen veri setinde EMA (5, 13, 21) hesaplayarak yeni sütunlar ekler.
    """
    data['EMA5'] = data['close'].ewm(span=5, adjust=False).mean()
    data['EMA13'] = data['close'].ewm(span=13, adjust=False).mean()
    data['EMA21'] = data['close'].ewm(span=21, adjust=False).mean()
    return data

def check_conditions(data):
    """
    Belirtilen EMA sıralaması koşulunu kontrol eder:
    EMA5 > EMA13 > EMA21
    """
    latest = data.iloc[-1]
    return (
        latest['EMA5'] > latest['EMA13'] and
        latest['EMA13'] > latest['EMA21']
    )

def main():
    """
    TV BIST verisi al.
    """
    all_symbols = get_all_symbols(market='turkey')
    bist_symbols = [symbol for symbol in all_symbols if 'BIST:' in symbol]

    results = []

    for symbol in bist_symbols:
        retry_count = 0
        max_retries = 2
        while retry_count <= max_retries:
            try:
                exchange, symbol_code = symbol.split(':')
                data = tv.get_hist(
                    symbol=symbol_code, exchange=exchange,
                    interval=Interval.in_daily, n_bars=100
                )
                data = data.reset_index()
                data = calculate_ema(data)

                if check_conditions(data):
                    latest_row = data.iloc[-1]
                    results.append({
                        'Symbol': symbol,
                        'Last Price': latest_row['close'],
                        'Date': latest_row['datetime']
                    })
                    print(f"{symbol}: Koşulları SAĞLIYOR")
                else:
                    print(f"{symbol}: Koşulları SAĞLAMIYOR")

                logger.info(f"Processed {symbol}")
                break
            except Exception as e:
                retry_count += 1
                logger.error(f"Error fetching data for {symbol} (Attempt {retry_count}/{max_retries}): {e}")
                time.sleep(2)
                if retry_count > max_retries:
                    logger.error(f"Max retries reached for {symbol}. Skipping...")
                    break

    # Sonuçları göster
    if results:
        results_df = pd.DataFrame(results)
        print("\nKoşulları Sağlayan Tarama Sonuçları:")
        print(results_df.to_string(index=False))
    else:
        print("Koşulları sağlayan sembol bulunamadı.")

if __name__ == "__main__":
    main()


ERROR:tvDatafeed.main:error while signin


BIST:KSTUR: Koşulları SAĞLAMIYOR
BIST:GUNDG: Koşulları SAĞLAMIYOR
BIST:GUBRF: Koşulları SAĞLAMIYOR
BIST:SKYLP: Koşulları SAĞLAMIYOR
BIST:INGRM: Koşulları SAĞLAMIYOR
BIST:AVGYO: Koşulları SAĞLIYOR
BIST:ARTMS: Koşulları SAĞLAMIYOR
BIST:SEKFK: Koşulları SAĞLAMIYOR
BIST:EUYO: Koşulları SAĞLAMIYOR
BIST:YBTAS: Koşulları SAĞLAMIYOR
BIST:PEHOL: Koşulları SAĞLAMIYOR
BIST:MARKA: Koşulları SAĞLIYOR
BIST:SEKUR: Koşulları SAĞLIYOR
BIST:QNBFL: Koşulları SAĞLAMIYOR
BIST:KMPUR: Koşulları SAĞLAMIYOR
BIST:ALKLC: Koşulları SAĞLIYOR
BIST:DOBUR: Koşulları SAĞLAMIYOR
BIST:BORSK: Koşulları SAĞLAMIYOR
BIST:ALKIM: Koşulları SAĞLAMIYOR
BIST:MAALT: Koşulları SAĞLIYOR
BIST:KONYA: Koşulları SAĞLAMIYOR
BIST:AKFGY: Koşulları SAĞLAMIYOR
BIST:EUREN: Koşulları SAĞLAMIYOR
BIST:SELEC: Koşulları SAĞLAMIYOR
BIST:METUR: Koşulları SAĞLIYOR
BIST:DESA: Koşulları SAĞLAMIYOR
BIST:KUVVA: Koşulları SAĞLAMIYOR
BIST:TGSAS: Koşulları SAĞLAMIYOR
BIST:ALCAR: Koşulları SAĞLAMIYOR
BIST:BANVT: Koşulları SAĞLAMIYOR
BIST:YYAPI: Koşulları SA

---
**Hadi biraz sonuçları daraltalım , potansiyel 5-13 kesişime yakın olanlara bakalım...**

In [9]:
import numpy as np
import pandas as pd
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
import warnings
import logging

# Uyarıları ve log ayarlarını yapılandırma
warnings.simplefilter(action='ignore')
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

# TradingView verilerini indirmek için kimlik bilgileriyle oturum açma
tv = TvDatafeed(username='your_username', password='your_password')


def calculate_ema(data):
    """
    EMA (5, 13, 21)
    """
    data['EMA5'] = data['close'].ewm(span=5, adjust=False).mean()
    data['EMA13'] = data['close'].ewm(span=13, adjust=False).mean()
    data['EMA21'] = data['close'].ewm(span=21, adjust=False).mean()
    return data


def check_conditions(data, proximity_threshold=0.005):
    """
    Belirtilen EMA sıralaması koşulunu, EMA 5'in EMA 13'e yakınlığını, son birkaç gün boyunca EMA 5'in EMA 13'ün
    altında olup olmadığını ve şu anda yukarı yönlü olup olmadığını kontrol eder:
    EMA5 > EMA13 > EMA21 ve EMA5 ile EMA13 arasındaki fark threshold değerine yakın
    """
    latest = data.iloc[-1]
    prev1 = data.iloc[-2]
    prev2 = data.iloc[-3]

    # EMA 5 ile EMA 13 yakınlığı
    ema_proximity = abs(latest['EMA5'] - latest['EMA13']) / latest['EMA13']

    # EMA 5'in son günlerde EMA 13'ün altında olup şimdi yukarı yönlü olması
    ema5_rising_and_near_cross = (
        latest['EMA5'] > latest['EMA13'] and
        prev1['EMA5'] < prev1['EMA13'] and
        prev2['EMA5'] < prev2['EMA13']
    )

    return (
        latest['EMA5'] > latest['EMA13'] and
        latest['EMA13'] > latest['EMA21'] and
        ema_proximity < proximity_threshold and
        ema5_rising_and_near_cross
    )


def main():
    """
    TV BIST verisi al.
    """
    all_symbols = get_all_symbols(market='turkey')
    bist_symbols = [symbol for symbol in all_symbols if 'BIST:' in symbol]

    results = []

    for symbol in bist_symbols:
        try:
            exchange, symbol_code = symbol.split(':')
            data = tv.get_hist(
                symbol=symbol_code, exchange=exchange,
                interval=Interval.in_daily, n_bars=100
            )
            data = data.reset_index()
            data = calculate_ema(data)

            if check_conditions(data):
                latest_row = data.iloc[-1]
                results.append({
                    'Symbol': symbol,
                    'Last Price': latest_row['close'],
                    'Date': latest_row['datetime'],
                    'EMA5': latest_row['EMA5'],
                    'EMA13': latest_row['EMA13']
                })
                print(f"{symbol}: Koşulları SAĞLIYOR (EMA5, EMA13'ü kesmeye yakın ve yükseliyor)")
            else:
                print(f"{symbol}: Koşulları SAĞLAMIYOR")

            logger.info(f"Processed {symbol}")
        except Exception as e:
            logger.error(f"Error fetching data for {symbol}: {e}")

    # Sonuçları göster
    if results:
        results_df = pd.DataFrame(results)
        print("\nKoşulları Sağlayan Tarama Sonuçları (EMA5, EMA13'ü Kesme Potansiyeline Sahip):")
        print(results_df.to_string(index=False))
    else:
        print("Koşulları sağlayan sembol bulunamadı.")


if __name__ == "__main__":
    main()


ERROR:tvDatafeed.main:error while signin


BIST:ALVES: Koşulları SAĞLAMIYOR
BIST:ZPLIB: Koşulları SAĞLAMIYOR
BIST:MERCN: Koşulları SAĞLAMIYOR
BIST:A1CAP: Koşulları SAĞLAMIYOR
BIST:DURDO: Koşulları SAĞLIYOR (EMA5, EMA13'ü kesmeye yakın ve yükseliyor)
BIST:IHEVA: Koşulları SAĞLAMIYOR
BIST:YIGIT: Koşulları SAĞLAMIYOR
BIST:AEFES: Koşulları SAĞLAMIYOR
BIST:DAPGM: Koşulları SAĞLAMIYOR
BIST:SNICA: Koşulları SAĞLAMIYOR
BIST:KERVT: Koşulları SAĞLAMIYOR
BIST:AKMGY: Koşulları SAĞLAMIYOR
BIST:BRISA: Koşulları SAĞLAMIYOR
BIST:EMKEL: Koşulları SAĞLAMIYOR
BIST:PNSUT: Koşulları SAĞLAMIYOR
BIST:YAYLA: Koşulları SAĞLAMIYOR
BIST:TSPOR: Koşulları SAĞLAMIYOR
BIST:KAYSE: Koşulları SAĞLAMIYOR
BIST:QUAGR: Koşulları SAĞLAMIYOR
BIST:ERCB: Koşulları SAĞLAMIYOR
BIST:HKTM: Koşulları SAĞLAMIYOR
BIST:TSGYO: Koşulları SAĞLAMIYOR
BIST:GRNYO: Koşulları SAĞLAMIYOR
BIST:ATEKS: Koşulları SAĞLAMIYOR
BIST:IHAAS: Koşulları SAĞLAMIYOR
BIST:LIDFA: Koşulları SAĞLAMIYOR
BIST:BOSSA: Koşulları SAĞLAMIYOR
BIST:GWIND: Koşulları SAĞLAMIYOR
BIST:ATSYH: Koşulları SAĞLAMIYOR
BIS



---


**Hadi fikir sahibi olmanız için taramalar yazayım şu anda aklıma gelen farklı kriterlerle kodlama örnekleri olsun :) Neleri baz aldım kod içerisinde açıklamalarda var.**

---



In [ ]:
import numpy as np
import pandas as pd
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
import ta
import warnings
import logging

# Uyarıları ve log ayarlarını yapılandırma
warnings.simplefilter(action='ignore')
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

# TradingView verilerini indirmek için kimlik bilgileriyle oturum açma
tv = TvDatafeed(username='your_username', password='your_password')


def calculate_indicators(data):
    """
    EMA (10, 20, 50), hacim ortalaması, RSI ve 50 günlük en düşük fiyat
    """
    data['EMA10'] = data['close'].ewm(span=10, adjust=False).mean()
    data['EMA20'] = data['close'].ewm(span=20, adjust=False).mean()
    data['EMA50'] = data['close'].ewm(span=50, adjust=False).mean()

    data['Volume_MA10'] = data['volume'].rolling(window=10).mean()
    data['RSI'] = ta.momentum.RSIIndicator(data['close'], window=14).rsi()

    data['Lowest50'] = data['low'].rolling(window=50).min()

    return data


def check_conditions(data):

    latest = data.iloc[-1]
    prev1 = data.iloc[-2]

    # EMA dönüş sinyali: EMA 10 EMA 20'ye yakın, EMA 50 ise EMA 20'nin biraz altında veya eşit
    ema_recovery_signal = latest['EMA10'] > latest['EMA20'] and (latest['EMA20'] >= latest['EMA50'] * 0.95)

    # RSI’nin 35-60 aralığında olup yükselişte olması
    rsi_recovery_potential = 35 < latest['RSI'] < 60 and latest['RSI'] > prev1['RSI']

    # Artan hacim
    rising_volume = latest['volume'] > latest['Volume_MA10']

    # Fiyatın 50 günlük en düşük seviyenin %10 üzerinde olması
    above_support_level = latest['close'] >= (latest['Lowest50'] * 1.10)

    return ema_recovery_signal and rsi_recovery_potential and rising_volume and above_support_level


def main():
    """
    TV BIST verisi al.
    """

    all_symbols = get_all_symbols(market='turkey')
    bist_symbols = [symbol for symbol in all_symbols if 'BIST:' in symbol]

    results = []

    for symbol in bist_symbols:
        try:
            exchange, symbol_code = symbol.split(':')
            data = tv.get_hist(
                symbol=symbol_code, exchange=exchange,
                interval=Interval.in_daily, n_bars=100
            )
            data = data.reset_index()
            data = calculate_indicators(data)

            if check_conditions(data):
                latest_row = data.iloc[-1]
                results.append({
                    'Symbol': symbol,
                    'Last Price': latest_row['close'],
                    'Date': latest_row['datetime'],
                    'EMA10': latest_row['EMA10'],
                    'EMA20': latest_row['EMA20'],
                    'EMA50': latest_row['EMA50'],
                    'Volume': latest_row['volume'],
                    'RSI': latest_row['RSI'],
                    'Lowest50': latest_row['Lowest50']
                })
                print(f"{symbol}: Koşulları SAĞLIYOR (Toparlanma Belirtileri ve Yükseliş Potansiyeli)")
            else:
                print(f"{symbol}: Koşulları SAĞLAMIYOR")

            logger.info(f"Processed {symbol}")
        except Exception as e:
            logger.error(f"Error fetching data for {symbol}: {e}")

    if results:
        results_df = pd.DataFrame(results)
        print("\nKoşulları Sağlayan Tarama Sonuçları (Toparlanma Belirtileri ve Yükseliş Potansiyeli):")
        print(results_df.to_string(index=False))
    else:
        print("Koşulları sağlayan sembol bulunamadı.")


if __name__ == "__main__":
    main()


ERROR:tvDatafeed.main:error while signin


BIST:METRO: Koşulları SAĞLAMIYOR
BIST:ORMA: Koşulları SAĞLAMIYOR
BIST:OBASE: Koşulları SAĞLAMIYOR
BIST:YIGIT: Koşulları SAĞLAMIYOR
BIST:IEYHO: Koşulları SAĞLAMIYOR
BIST:ALKIM: Koşulları SAĞLAMIYOR
BIST:KLGYO: Koşulları SAĞLAMIYOR
BIST:MAALT: Koşulları SAĞLAMIYOR
BIST:GEDIK: Koşulları SAĞLAMIYOR
BIST:KERVT: Koşulları SAĞLAMIYOR
BIST:KAYSE: Koşulları SAĞLAMIYOR
BIST:EFORC: Koşulları SAĞLAMIYOR
BIST:USAK: Koşulları SAĞLAMIYOR
BIST:SEKFK: Koşulları SAĞLAMIYOR
BIST:Z30KE: Koşulları SAĞLAMIYOR
BIST:BORSK: Koşulları SAĞLAMIYOR
BIST:GUNDG: Koşulları SAĞLAMIYOR
BIST:RODRG: Koşulları SAĞLAMIYOR
BIST:ODINE: Koşulları SAĞLAMIYOR
BIST:YEOTK: Koşulları SAĞLAMIYOR
BIST:HEDEF: Koşulları SAĞLAMIYOR
BIST:LYDHO: Koşulları SAĞLAMIYOR
BIST:CRFSA: Koşulları SAĞLIYOR (Toparlanma Belirtileri ve Yükseliş Potansiyeli)
BIST:MEPET: Koşulları SAĞLAMIYOR
BIST:AKFYE: Koşulları SAĞLAMIYOR
BIST:AYDEM: Koşulları SAĞLAMIYOR
BIST:BRKSN: Koşulları SAĞLAMIYOR
BIST:DITAS: Koşulları SAĞLAMIYOR
BIST:TUCLK: Koşulları SAĞLAMIYO

In [10]:
import numpy as np
import pandas as pd
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
import ta
import warnings
import logging

# Uyarıları ve log ayarlarını yapılandırma
warnings.simplefilter(action='ignore')
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

# TradingView verilerini indirmek için kimlik bilgileriyle oturum açma
tv = TvDatafeed(username='your_username', password='your_password')


def calculate_indicators(data):
    """
    EMA (10, 20, 50), hacim ortalaması, RSI ve 50 günlük en düşük fiyat
    """
    data['EMA10'] = data['close'].ewm(span=10, adjust=False).mean()
    data['EMA20'] = data['close'].ewm(span=20, adjust=False).mean()
    data['EMA50'] = data['close'].ewm(span=50, adjust=False).mean()

    data['Volume_MA5'] = data['volume'].rolling(window=5).mean()
    data['RSI'] = ta.momentum.RSIIndicator(data['close'], window=14).rsi()
    data['Lowest50'] = data['low'].rolling(window=50).min()

    return data


def check_conditions(data):
    """
    Düşüş trendini kıran ve potansiyel yükseliş işareti gösteren varlıklara bakalım.
    """
    latest = data.iloc[-1]
    prev1 = data.iloc[-2]

    # Düşüş trendini kıran kısa vadeli EMA sinyali: EMA 10 > EMA 20
    trend_break_signal = latest['EMA10'] > latest['EMA20'] and latest['close'] < latest['EMA50']

    # Dipten toparlanma işareti: Fiyatın son 50 günün en düşüğünün %10 üzerinde olması
    above_support_level = latest['close'] >= (latest['Lowest50'] * 1.10)

    # Momentum onayı: RSI’nin 45-60 aralığında yükseliyor olması
    rsi_recovery_potential = 45 < latest['RSI'] < 60 and latest['RSI'] > prev1['RSI']

    # Artan işlem hacmi
    rising_volume = latest['volume'] > latest['Volume_MA5']

    return trend_break_signal and above_support_level and rsi_recovery_potential and rising_volume


def main():
    """
    TV BIST verisi al.
    """
    all_symbols = get_all_symbols(market='turkey')
    bist_symbols = [symbol for symbol in all_symbols if 'BIST:' in symbol]

    results = []

    for symbol in bist_symbols:
        try:
            exchange, symbol_code = symbol.split(':')
            data = tv.get_hist(
                symbol=symbol_code, exchange=exchange,
                interval=Interval.in_daily, n_bars=100
            )

            # Verilerin boş olup olmadığını kontrol edin
            if data is None or data.empty:
                logger.warning(f"No data for {symbol}, skipping...")
                continue

            data = data.reset_index()
            data = calculate_indicators(data)

            if check_conditions(data):
                latest_row = data.iloc[-1]
                results.append({
                    'Symbol': symbol,
                    'Last Price': latest_row['close'],
                    'Date': latest_row['datetime'],
                    'EMA10': latest_row['EMA10'],
                    'EMA20': latest_row['EMA20'],
                    'EMA50': latest_row['EMA50'],
                    'Volume': latest_row['volume'],
                    'RSI': latest_row['RSI'],
                    'Lowest50': latest_row['Lowest50']
                })
                print(f"{symbol}: Koşulları SAĞLIYOR (Düşeni Kıran Potansiyel Yükseliş)")
            else:
                print(f"{symbol}: Koşulları SAĞLAMIYOR")

            logger.info(f"Processed {symbol}")
        except Exception as e:
            logger.error(f"Error fetching data for {symbol}: {e}")

    # Sonuçları göster
    if results:
        results_df = pd.DataFrame(results)
        print("\nKoşulları Sağlayan Tarama Sonuçları (Düşeni Kıran Potansiyel Yükseliş):")
        print(results_df.to_string(index=False))
    else:
        print("Koşulları sağlayan sembol bulunamadı.")


if __name__ == "__main__":
    main()


ERROR:tvDatafeed.main:error while signin


BIST:EDATA: Koşulları SAĞLAMIYOR
BIST:YEOTK: Koşulları SAĞLAMIYOR
BIST:EMKEL: Koşulları SAĞLAMIYOR
BIST:DERIM: Koşulları SAĞLAMIYOR
BIST:IHLGM: Koşulları SAĞLAMIYOR
BIST:RYGYO: Koşulları SAĞLAMIYOR
BIST:LINK: Koşulları SAĞLAMIYOR
BIST:TCKRC: Koşulları SAĞLAMIYOR
BIST:DZGYO: Koşulları SAĞLAMIYOR
BIST:USDTR: Koşulları SAĞLAMIYOR
BIST:BIENY: Koşulları SAĞLAMIYOR
BIST:FENER: Koşulları SAĞLAMIYOR
BIST:ZPLIB: Koşulları SAĞLAMIYOR
BIST:ISBIR: Koşulları SAĞLAMIYOR
BIST:BEGYO: Koşulları SAĞLAMIYOR
BIST:DNISI: Koşulları SAĞLAMIYOR
BIST:MERIT: Koşulları SAĞLAMIYOR
BIST:CEMAS: Koşulları SAĞLAMIYOR
BIST:LKMNH: Koşulları SAĞLAMIYOR
BIST:HUNER: Koşulları SAĞLAMIYOR
BIST:MAGEN: Koşulları SAĞLAMIYOR
BIST:RALYH: Koşulları SAĞLAMIYOR
BIST:BORSK: Koşulları SAĞLAMIYOR
BIST:TABGD: Koşulları SAĞLAMIYOR
BIST:BAGFS: Koşulları SAĞLAMIYOR
BIST:QUAGR: Koşulları SAĞLAMIYOR
BIST:ISMEN: Koşulları SAĞLAMIYOR
BIST:KONKA: Koşulları SAĞLAMIYOR
BIST:PRDGS: Koşulları SAĞLAMIYOR
BIST:YKSLN: Koşulları SAĞLAMIYOR
BIST:SONME:

In [11]:
import numpy as np
import pandas as pd
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
import ta
import warnings
import logging

# Uyarıları ve log ayarlarını yapılandırma
warnings.simplefilter(action='ignore')
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

# TradingView verilerini indirmek için kimlik bilgileriyle oturum açma
tv = TvDatafeed(username='your_username', password='your_password')


def calculate_indicators(data):
    """
    Hacim ortalaması, RSI ve 50 günlük en düşük fiyat bakalım.
    """
    data['Volume_MA5'] = data['volume'].rolling(window=5).mean()
    data['RSI'] = ta.momentum.RSIIndicator(data['close'], window=14).rsi()
    data['Lowest50'] = data['low'].rolling(window=50).min()

    return data


def check_conditions(data):
    """
    Güçlü destekten dönüş sinyali ve hacim artışı gösteren varlıklara bakalım.
    """
    latest = data.iloc[-1]
    prev1 = data.iloc[-2]
    prev2 = data.iloc[-3]

    # Fiyatın destekten dönüş işareti: Son 50 günün en düşüğünün %5-10 üzerinde olmalı
    above_support_level = latest['close'] >= (latest['Lowest50'] * 1.05) and latest['close'] <= (latest['Lowest50'] * 1.10)

    # Fiyatın iki gün üst üste yükselmesi (toparlanma işareti)
    price_recovery_signal = latest['close'] > prev1['close'] > prev2['close']

    # Artan işlem hacmi: Hacim, 5 günlük ortalamanın %50 üzerinde
    rising_volume = latest['volume'] > 1.5 * latest['Volume_MA5']

    # RSI’nin 40-60 aralığında olup yükseliş eğiliminde olması
    stable_rsi = 40 < latest['RSI'] < 60 and latest['RSI'] > prev1['RSI']

    return above_support_level and price_recovery_signal and rising_volume and stable_rsi


def main():
    """
    TV BIST verisi al.
    """
    all_symbols = get_all_symbols(market='turkey')
    bist_symbols = [symbol for symbol in all_symbols if 'BIST:' in symbol]

    results = []

    for symbol in bist_symbols:
        try:
            exchange, symbol_code = symbol.split(':')
            data = tv.get_hist(
                symbol=symbol_code, exchange=exchange,
                interval=Interval.in_daily, n_bars=100
            )

            # Verilerin boş olup olmadığını kontrol edin
            if data is None or data.empty:
                logger.warning(f"No data for {symbol}, skipping...")
                continue

            data = data.reset_index()
            data = calculate_indicators(data)

            if check_conditions(data):
                latest_row = data.iloc[-1]
                results.append({
                    'Symbol': symbol,
                    'Last Price': latest_row['close'],
                    'Date': latest_row['datetime'],
                    'Volume': latest_row['volume'],
                    'RSI': latest_row['RSI'],
                    'Lowest50': latest_row['Lowest50']
                })
                print(f"{symbol}: Koşulları SAĞLIYOR (Güçlü Destekten Dönüş)")
            else:
                print(f"{symbol}: Koşulları SAĞLAMIYOR")

            logger.info(f"Processed {symbol}")
        except Exception as e:
            logger.error(f"Error fetching data for {symbol}: {e}")

    # Sonuçları göster
    if results:
        results_df = pd.DataFrame(results)
        print("\nKoşulları Sağlayan Tarama Sonuçları (Güçlü Destekten Dönüş):")
        print(results_df.to_string(index=False))
    else:
        print("Koşulları sağlayan sembol bulunamadı.")


if __name__ == "__main__":
    main()


ERROR:tvDatafeed.main:error while signin


BIST:IZENR: Koşulları SAĞLAMIYOR
BIST:IHLAS: Koşulları SAĞLAMIYOR
BIST:OFSYM: Koşulları SAĞLAMIYOR
BIST:EGSER: Koşulları SAĞLAMIYOR
BIST:BJKAS: Koşulları SAĞLAMIYOR
BIST:INDES: Koşulları SAĞLAMIYOR
BIST:BINBN: Koşulları SAĞLAMIYOR
BIST:BERA: Koşulları SAĞLAMIYOR
BIST:IHYAY: Koşulları SAĞLAMIYOR
BIST:SEGMN: Koşulları SAĞLAMIYOR
BIST:YIGIT: Koşulları SAĞLAMIYOR
BIST:DURDO: Koşulları SAĞLAMIYOR
BIST:LIDER: Koşulları SAĞLAMIYOR
BIST:PSDTC: Koşulları SAĞLAMIYOR
BIST:KRONT: Koşulları SAĞLAMIYOR
BIST:MNDTR: Koşulları SAĞLAMIYOR
BIST:DENGE: Koşulları SAĞLAMIYOR
BIST:SKYMD: Koşulları SAĞLAMIYOR
BIST:CEMTS: Koşulları SAĞLAMIYOR
BIST:AVHOL: Koşulları SAĞLAMIYOR
BIST:KRDMA: Koşulları SAĞLAMIYOR
BIST:SELVA: Koşulları SAĞLAMIYOR
BIST:MARTI: Koşulları SAĞLAMIYOR
BIST:AGYO: Koşulları SAĞLAMIYOR
BIST:PENGD: Koşulları SAĞLAMIYOR
BIST:AKSEN: Koşulları SAĞLAMIYOR
BIST:DOHOL: Koşulları SAĞLAMIYOR
BIST:DESPC: Koşulları SAĞLAMIYOR
BIST:NTGAZ: Koşulları SAĞLAMIYOR
BIST:SILVR: Koşulları SAĞLAMIYOR
BIST:OZKGY: 

In [ ]:
import numpy as np
import pandas as pd
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
import ta
import warnings
import logging

# Uyarıları ve log ayarlarını yapılandırma
warnings.simplefilter(action='ignore')
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

# TradingView verilerini indirmek için kimlik bilgileriyle oturum açma
tv = TvDatafeed(username='your_username', password='your_password')


def calculate_indicators(data):
    """
    Verilen veri setinde EMA (10, 20), hacim ortalaması, RSI ve 20 günlük en düşük fiyatı hesaplayarak yeni sütunlar ekler.
    """
    data['EMA10'] = data['close'].ewm(span=10, adjust=False).mean()
    data['EMA20'] = data['close'].ewm(span=20, adjust=False).mean()
    data['Volume_MA10'] = data['volume'].rolling(window=10).mean()
    data['RSI'] = ta.momentum.RSIIndicator(data['close'], window=14).rsi()
    data['Lowest20'] = data['low'].rolling(window=20).min()

    return data


def check_conditions(data):
    """
    Momentum ve trend yakalama sinyali gösteren varlıkları belirlemek için koşulları kontrol eder.
    """
    latest = data.iloc[-1]
    prev1 = data.iloc[-2]

    # Kısa Vadeli Momentum Artışı: RSI 50’nin üzerinde ve son iki gün artıyor
    momentum_rising = latest['RSI'] > 50 and latest['RSI'] > prev1['RSI']

    # Güçlü Kısa Vadeli Trend: EMA 10, EMA 20'nin üzerinde
    short_term_trend = latest['EMA10'] > latest['EMA20']

    # Destekten Toparlanma: Fiyat son 20 günün en düşüğünün %10 üzerinde
    support_rebound = latest['close'] >= (latest['Lowest20'] * 1.10)

    # İşlem hacminin 10 günlük ortalamanın %50 üzerinde olması
    volume_confirmation = latest['volume'] > 1.5 * latest['Volume_MA10']

    return momentum_rising and short_term_trend and support_rebound and volume_confirmation


def main():
    """
    TV BIST verisi al.
    """
    all_symbols = get_all_symbols(market='turkey')
    bist_symbols = [symbol for symbol in all_symbols if 'BIST:' in symbol]

    results = []

    for symbol in bist_symbols:
        try:
            exchange, symbol_code = symbol.split(':')
            data = tv.get_hist(
                symbol=symbol_code, exchange=exchange,
                interval=Interval.in_daily, n_bars=100
            )

            # Verilerin boş olup olmadığını kontrol edin
            if data is None or data.empty:
                logger.warning(f"No data for {symbol}, skipping...")
                continue

            data = data.reset_index()
            data = calculate_indicators(data)

            if check_conditions(data):
                latest_row = data.iloc[-1]
                results.append({
                    'Symbol': symbol,
                    'Last Price': latest_row['close'],
                    'Date': latest_row['datetime'],
                    'EMA10': latest_row['EMA10'],
                    'EMA20': latest_row['EMA20'],
                    'Volume': latest_row['volume'],
                    'RSI': latest_row['RSI'],
                    'Lowest20': latest_row['Lowest20']
                })
                print(f"{symbol}: Koşulları SAĞLIYOR (Momentum ve Trend Yakalama)")
            else:
                print(f"{symbol}: Koşulları SAĞLAMIYOR")

            logger.info(f"Processed {symbol}")
        except Exception as e:
            logger.error(f"Error fetching data for {symbol}: {e}")

    # Sonuçları göster
    if results:
        results_df = pd.DataFrame(results)
        print("\nKoşulları Sağlayan Tarama Sonuçları (Momentum ve Trend Yakalama):")
        print(results_df.to_string(index=False))
    else:
        print("Koşulları sağlayan sembol bulunamadı.")


if __name__ == "__main__":
    main()


In [ ]:
import numpy as np
import pandas as pd
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
import ta
import warnings
import logging

# Uyarıları ve log ayarlarını yapılandırma
warnings.simplefilter(action='ignore')
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

# TradingView verilerini indirmek için kimlik bilgileriyle oturum açma
tv = TvDatafeed(username='your_username', password='your_password')


def calculate_indicators(data):
    """
    Verilen veri setinde hacim ortalaması, RSI, Stokastik Osilatör ve 50 günlük en düşük fiyatı hesaplayarak yeni sütunlar ekler.
    """
    data['Volume_MA5'] = data['volume'].rolling(window=5).mean()
    data['RSI'] = ta.momentum.RSIIndicator(data['close'], window=14).rsi()

    stoch = ta.momentum.StochasticOscillator(data['high'], data['low'], data['close'], window=14, smooth_window=3)
    data['Stoch_%K'] = stoch.stoch()
    data['Stoch_%D'] = stoch.stoch_signal()

    data['Lowest50'] = data['low'].rolling(window=50).min()

    return data


def check_conditions(data):
    """
    Toparlanma ve hacim artışı sinyali gösteren varlıkları belirlemek için koşulları kontrol eder.
    """
    latest = data.iloc[-1]
    prev1 = data.iloc[-2]

    # RSI'nin 40'ın üzerinde olup son iki gün yükseliyor olması
    rsi_recovery_signal = latest['RSI'] > 40 and latest['RSI'] > prev1['RSI']

    # Stokastik Osilatör %K'nin %D'yi yukarıdan kesmesi
    stoch_recovery_signal = latest['Stoch_%K'] > latest['Stoch_%D']

    # Artan işlem hacmi: Hacim 5 günlük ortalamanın üzerinde
    rising_volume = latest['volume'] > latest['Volume_MA5']

    # Fiyatın son 50 günün en düşüğünün %5 üzerinde olması
    above_support_level = latest['close'] >= (latest['Lowest50'] * 1.05)

    return rsi_recovery_signal and stoch_recovery_signal and rising_volume and above_support_level


def main():
    """
    TV BIST verisi al.
    """
    all_symbols = get_all_symbols(market='turkey')
    bist_symbols = [symbol for symbol in all_symbols if 'BIST:' in symbol]

    results = []

    for symbol in bist_symbols:
        try:
            exchange, symbol_code = symbol.split(':')
            data = tv.get_hist(
                symbol=symbol_code, exchange=exchange,
                interval=Interval.in_daily, n_bars=100
            )

            # Verilerin boş olup olmadığını kontrol edin
            if data is None or data.empty:
                logger.warning(f"No data for {symbol}, skipping...")
                continue

            data = data.reset_index()
            data = calculate_indicators(data)

            if check_conditions(data):
                latest_row = data.iloc[-1]
                results.append({
                    'Symbol': symbol,
                    'Last Price': latest_row['close'],
                    'Date': latest_row['datetime'],
                    'Volume': latest_row['volume'],
                    'RSI': latest_row['RSI'],
                    'Stoch_%K': latest_row['Stoch_%K'],
                    'Stoch_%D': latest_row['Stoch_%D'],
                    'Lowest50': latest_row['Lowest50']
                })
                print(f"{symbol}: Koşulları SAĞLIYOR (Toparlanma ve Hacim Artışı)")
            else:
                print(f"{symbol}: Koşulları SAĞLAMIYOR")

            logger.info(f"Processed {symbol}")
        except Exception as e:
            logger.error(f"Error fetching data for {symbol}: {e}")

    # Sonuçları göster
    if results:
        results_df = pd.DataFrame(results)
        print("\nKoşulları Sağlayan Tarama Sonuçları (Toparlanma ve Hacim Artışı):")
        print(results_df.to_string(index=False))
    else:
        print("Koşulları sağlayan sembol bulunamadı.")


if __name__ == "__main__":
    main()


In [ ]:
import numpy as np
import pandas as pd
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
import ta
import warnings
import logging

# Uyarıları ve log ayarlarını yapılandırma
warnings.simplefilter(action='ignore')
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

# TradingView verilerini indirmek için kimlik bilgileriyle oturum açma
tv = TvDatafeed(username='your_username', password='your_password')


def calculate_indicators(data):
    """
    Hacim ortalaması, RSI, Stokastik Osilatör ve 50 günlük en düşük fiyat.
    """
    data['Volume_MA5'] = data['volume'].rolling(window=5).mean()
    data['RSI'] = ta.momentum.RSIIndicator(data['close'], window=14).rsi()

    stoch = ta.momentum.StochasticOscillator(data['high'], data['low'], data['close'], window=14, smooth_window=3)
    data['Stoch_%K'] = stoch.stoch()
    data['Stoch_%D'] = stoch.stoch_signal()

    data['Lowest50'] = data['low'].rolling(window=50).min()

    return data


def score_conditions(latest, prev1):
    """
    Verilen en son veri noktaları için puanlama.
    """
    score = 0

    # RSI Yükselişi Puanı (30 Puan)
    rsi_increase = latest['RSI'] - prev1['RSI']
    if rsi_increase > 0:
        score += min(30, rsi_increase * 1.5)  # Her %1 artış için 1.5 puan

    # Stokastik Osilatör Puanı (30 Puan)
    if latest['Stoch_%K'] > latest['Stoch_%D']:
        stoch_score = min(30, latest['Stoch_%K'] - latest['Stoch_%D'])
        score += stoch_score

    # Hacim Artışı Puanı (20 Puan)
    volume_increase = latest['volume'] / latest['Volume_MA5']
    if volume_increase > 1:
        score += min(20, (volume_increase - 1) * 10)

    # Destek Seviyesinden Uzaklık Puanı (20 Puan)
    support_distance = (latest['close'] - latest['Lowest50']) / latest['Lowest50']
    if support_distance > 0:
        score += min(20, support_distance * 10)

    return score


def main():
    """
    TV BIST verisi al.
    """
    # Kaç sonuç gösterilecek
    try:
        results_to_show = int(input("Kaç sonuç gösterilsin? (Varsayılan: 10): ") or 10)
    except ValueError:
        print("Geçersiz giriş. Varsayılan değer olan 10 kullanılacak.")
        results_to_show = 10

    all_symbols = get_all_symbols(market='turkey')
    bist_symbols = [symbol for symbol in all_symbols if 'BIST:' in symbol]

    results = []

    for symbol in bist_symbols:
        try:
            exchange, symbol_code = symbol.split(':')
            data = tv.get_hist(
                symbol=symbol_code, exchange=exchange,
                interval=Interval.in_daily, n_bars=100
            )

            # Verilerin boş olup olmadığını kontrol edin
            if data is None or data.empty:
                logger.warning(f"No data for {symbol}, skipping...")
                continue

            data = data.reset_index()
            data = calculate_indicators(data)

            latest = data.iloc[-1]
            prev1 = data.iloc[-2]

            score = score_conditions(latest, prev1)
            if score > 0:
                results.append({
                    'Symbol': symbol,
                    'Score': score,
                    'Last Price': latest['close'],
                    'Date': latest['datetime'],
                    'Volume': latest['volume'],
                    'RSI': latest['RSI'],
                    'Stoch_%K': latest['Stoch_%K'],
                    'Stoch_%D': latest['Stoch_%D'],
                    'Lowest50': latest['Lowest50']
                })
                print(f"{symbol}: Puan = {score:.2f} (Toparlanma ve Hacim Artışı)")
            else:
                print(f"{symbol}: Puanı Yetersiz")

            logger.info(f"Processed {symbol}")
        except Exception as e:
            logger.error(f"Error fetching data for {symbol}: {e}")

    # Sonuçları puana göre sıralama
    if results:
        results_df = pd.DataFrame(results).sort_values(by='Score', ascending=False)
        print("\nPuanlama Sonuçları (Toparlanma ve Hacim Artışı):")
        print(results_df.head(results_to_show).to_string(index=False))  # Sadece belirtilen sayıda sonuç gösterilir
    else:
        print("Koşulları sağlayan sembol bulunamadı.")


if __name__ == "__main__":
    main()
